## Setup

In [ ]:
# Hack the sys path for import
import sys
sys.path.insert(0,'..')

In [ ]:
import sqlite3
from database import Database

In [ ]:
db = Database("sql_terugkom_ochtend.db")

In [ ]:
# Clean up
db.query("DROP TABLE IF EXISTS Producten;")
db.query("DROP TABLE IF EXISTS ProductKenmerken;")

## Product eigenschappen: Algemeen vs specifiek

#### Producten tabel - Alleen algemene kenmerken

De producten tabel bevat alleen kenmerken die **ieder** product heeft; naam, merk, omschrijving, et cetera.

In [ ]:
db.query("""
    CREATE TABLE Producten (
        ProductId INTEGER,
        Naam TEXT,
        Merk TEXT,
        Omschrijving TEXT
    );
""")

db.query("""
    INSERT INTO Producten
    VALUES
        (1, "Samsung SSD 1TB", "Samsung", "Samsung - Solid State Disk - 1 TB - Laadt je data nog sneller!"),
        (2, "Samsung Monitor 24 inch", "Samsung", "Samsung Monitor - 24 inch - Supergoed scherm voor data scientist!")
    ;
""")

db.query("SELECT * FROM Producten")

#### Tabel ProductKenmerken: Specifieke kenmerken

Deze kenmerken kunnen verschillen per product (groep), daarom slaan we product, kenmerk en waarde op.
Hierdoor blijft het schema van de tabel gelijk voor alle soorten producten!

In [ ]:
db.query("""
    CREATE TABLE ProductKenmerken (
        ProductId INTEGER,
        Kenmerk TEXT,
        Waarde TEXT
    );
""")

db.query("""
    INSERT INTO ProductKenmerken
    VALUES
        (1, "Opslagruimte", "1 TB"),
        (1, "Formaat", "2.5 inch"),
        (1, "Aansluiting", "SATA-600"),
        (1, "Snelheid - Lezen", "3.400 MB/s"),
        (1, "Snelheid - Schrijven", "2.300 MB/s")
""")

db.query("SELECT * FROM ProductKenmerken")

#### Combineren algemene en specifieke kenmerken

In [ ]:
# Pivot in pandas; SQLite ondersteunt geen pivot
specifiek = (
    db.query("SELECT * FROM ProductKenmerken")
    .pivot(index="ProductId", columns="Kenmerk", values="Waarde")
)
specifiek

In [ ]:
# Haal algemene kenmerken op
algemeen = db.query("SELECT * FROM Producten")

# Combineer met specifieke kenmerken
algemeen.merge(specifiek, on="ProductId", how="left")

# SQLite API

In [ ]:
# Connectie maken; SQLite gebruikt een bestand
connection = sqlite3.connect("sql_terugkom_ochtend.db")

In [ ]:
# Maak een sqlite3.Cursor aan; deze houdt de status / het resultaat van een query bij
cursor = connection.cursor()
type(cursor)

In [ ]:
# Eigenschappen van de cursor
[_ for _ in dir(cursor) if not _.startswith("_")]

In [ ]:
# Voer een enkele query uit
cursor.execute("SELECT * FROM Producten")

In [ ]:
# Haal alle resultaten op
# Merk op: Lijst met tuples, veldnamen ontbreken...
cursor.fetchall()

In [ ]:
# Via row_factory kun je meer info ophalen
cursor.row_factory = sqlite3.Row


cursor.execute("SELECT * FROM Producten")
row = cursor.fetchone()

row

In [ ]:
# Namen van de kolommen
row.keys()

In [ ]:
# Enkele kolom ophalen
row["Omschrijving"]

In [ ]:
# Hele rij als dict
dict(row)

In [ ]:
# Alternatieve manier om kolom informatie te achterhalen
cursor.description

In [ ]:
connection.close()

## SQL Transacties

Door transacties te gebruiken kun je SQL queries "bundelen". Als een query faalt, kun je alle queries terugdraaien en de database in originele staat terugbrengen.

In [ ]:
# Maak connectie en cursor
connection = sqlite3.connect("sql_terugkom_ochtend.db")
cursor = connection.cursor()

In [ ]:
# Voer meerdere queries tegelijk uit
cursor.executescript("""
    DROP TABLE IF EXISTS Producten;
    DROP TABLE IF EXISTS ProductKenmerken;

    CREATE TABLE Producten (
        ProductId INTEGER PRIMARY KEY,
        Naam TEXT UNIQUE,
        Merk TEXT,
        Omschrijving TEXT
    );

    CREATE TABLE ProductKenmerken (
        ProductId INTEGER,
        Kenmerk TEXT,
        Waarde TEXT
    );
""")

In [ ]:
db.list_tables()

In [ ]:
# Gaat fout bij tweede keer draaien
try:
    
    cursor.executescript("""
        BEGIN TRANSACTION;

        INSERT INTO ProductKenmerken (ProductId, Kenmerk, Waarde)
        VALUES
            (1, 'Test Property A', 1),
            (1, 'Test Property B', 2),
            (1, 'Test Property C', 3)
        ;

        INSERT INTO Producten (Naam, Merk, Omschrijving)
            VALUES('Test Product', 'Test Merk', 'Omschrijving van het testproduct.');

        COMMIT;
    """)
    print("All done!")

except sqlite3.IntegrityError:
    
    print("On nee! Een dubbel product...")
    cursor.execute("ROLLBACK;")

In [ ]:
cursor.execute("SELECT * FROM Producten;").fetchall()

In [ ]:
cursor.execute("SELECT * FROM ProductKenmerken;").fetchall()

In [ ]:
connection.close()

## Upsert: Insert or Update

Upsert will try to insert a record, if a conflict occurs it will try to update the existing record.
The upsert operation only touches the conflicting row (unlike a regular update statement).


**Note: Not part of the SQL standard, each RDBMS will have its own implementation!**

In [ ]:
db = Database("sql_terugkom_ochtend.db")

In [ ]:
# Maak producten tabel aan
db.query("DROP TABLE IF EXISTS Products;")
db.query("""
    CREATE TABLE Products (
        id INTEGER PRIMARY KEY,
        Naam TEXT UNIQUE,
        Merk TEXT,
        Omschrijving TEXT,
        Versie INTEGER DEFAULT 1
    );
""")

In [ ]:
# Normaal INSERT statement
db.query("""
    INSERT INTO Products
        (Naam, Merk, Omschrijving)
        VALUES('Test Product 1', 'Test Merk', 'Verouderde omschrijving test product.')
""")
db.query("SELECT * FROM Products;")

In [ ]:
# Gebruik ON CONFLICT om UPDATE in plaats van INSERT uit te voeren!
db.query("""
    INSERT INTO Products (Naam, Merk, Omschrijving) VALUES
        ('Test Product 1', 'Test Merk', 'Omschrijving test product 1'),
        ('Test Product 2', 'Test Merk', 'Omschrijving test product 2')
        
    ON CONFLICT (Naam) DO UPDATE SET
        Naam = excluded.Naam,
        Merk = excluded.Merk,
        Omschrijving = excluded.Omschrijving,
        Versie = excluded.Versie + 1
    ;
""")

In [ ]:
db.query("SELECT * FROM Products;")

## Recursive queries

In [ ]:
db.query("DROP TABLE IF EXISTS Categories;")
db.query("""
    CREATE TABLE Categories (
        id INTEGER,
        parent INTEGER,
        label TEXT
    );
""")

In [ ]:
db.query("""
    INSERT INTO Categories (id, parent, label)
    VALUES
        (1, 0, 'Computers'),
        (2, 1, 'Harde schijven'),
        (3, 2, 'SSD'),
        (4, 2, 'HDD'),
        (5, 1, 'Monitoren'),
        (6, 0, 'Witgoed'),
        (7, 6, 'Wasmachines')
    ;
""")

In [ ]:
db.query("SELECT * FROM Categories;")

**Recursive query format:**

In [ ]:
# Items under main menu item
category = "Computers"
category = "Witgoed"

db.query(f"""
    WITH RECURSIVE under_item (id, label, level) AS (
            SELECT
                id,
                label,
                0
            FROM Categories
            WHERE label = '{category}'
        
        UNION ALL
        
        SELECT
            cat.id,
            cat.label,
            under_item.level + 1
        FROM Categories cat
        JOIN under_item ON cat.parent=under_item.id
    )
    SELECT * FROM under_item ORDER BY id
    ;
""")

In [ ]:
# Reverse query: Bread crumb trail
item = "SSD"
# item = "Monitoren"

db.query(f"""
    WITH RECURSIVE above_item (id, parent, label, level) AS (
            SELECT
                id,
                parent,
                label,
                0
            FROM Categories
            WHERE label = '{item}'
        
        UNION ALL
        
        SELECT
            cat.id,
            cat.parent,
            cat.label,
            above_item.level - 1
        FROM Categories cat
        JOIN above_item ON cat.id=above_item.parent
            AND cat.id >= 0
    )
    SELECT *
    FROM above_item
    ORDER BY level
    ;
""")

In [ ]:
# Clean up
db.query("DROP TABLE IF EXISTS Categories;")